In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from kmodes.kprototypes import KPrototypes
from scipy.optimize import curve_fit
import seaborn as sns



# INDEX

### 1- Processament i neteja de les dades
    1.1- Eliminar unitats de consum
    1.2- Variables temporals (continues)
        1.2.1- Shift horari. Posem l'origen a les 4a.m.
        1.2.2- Columnes estació de l'any i cap de setmana/entre setmana
    1.3- Selecció de columnes
    
 
    
       

# 1- Processament i neteja de les dades


    1. En primer lloc, realitzem un processament i neteja de les dades. Eliminem els registres corresponents a àudio i aquells que no tenen les variables contínues (hora de consum i durades) ben informades (NaN). Per les altres dimensions, tractem els valors NaN com una categoria més.
    
    2. Després, realitzem un shift horari de 4 hores per posar l'origen en una hora on el consum és baix, per raons de visualització.
    
    3. Afegim columnes de la data: una per el dia de la setmana (dilluns, dimarts...), un altre pel mes (gener, febrer...) i un altre per l'any (2019, 2020...). Ocasionalment, també agrupem aquestes columnes en dues de noves corresponents a entre setmana/cap de setmana i una altre amb l'estació de l'any. 
    

In [2]:
df = pd.read_csv('dades_consum_logats.csv')   # Llegim el fitxer de dades
df

C:\Users\josep\anaconda3\envs\ox\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9,15,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,bo_es_diferit,min_data,bi_segons_consum,usuari_id,player_id,an,font,media_tipus,producte_id,contingut_id,...,dispositiu_browser_nom,dispositiu_tipus,live,ui,provincia_id,dispositiu_accepta_cookies,usuari,mes,dia,hora
0,True,2019-01-01 00:01:23,224,cd59b5d0-eadd-11e7-8b91-7b6a96b37d13,532431546297264625,False,IP,video,PortalCCMA,5808597,...,CHROME,TABLET,diferit,MzQ1MDg1NDA2,NaN,True,MzQ1MDg1NDA2,1,1,0
1,True,2019-01-01 00:02:50,936,ef3a5e30-5a30-11e8-b285-6b1b1792d2d5,151871546297352716,False,IP,video,PortalCCMA,5809807,...,CHROME,MOBILE,diferit,MzQyODYxMjQ5,NaN,True,MzQyODYxMjQ5,1,1,0
2,True,2019-01-01 00:04:43,6821,971bc1a0-d617-11e7-9975-57f16f7e845d,94429,False,IP,video,appMobil_tv3,5808127,...,ANDROID BROWSER,MOBILE,diferit,MzQzMTM5MDQ4,NaN,True,MzQzMTM5MDQ4,1,1,0
3,True,2019-01-01 00:04:48,0,58169d60-b60c-11e8-85da-a1c496d066bd,16755,False,IP,video,appMobil_tv3,5809111,...,CHROME,MOBILE,diferit,MzQ0OTI4NTM1,NaN,True,MzQ0OTI4NTM1,1,1,0
4,True,2019-01-01 00:05:29,0,7c546040-0d50-11e9-b7c3-ab71693993b0,81949,False,IP,video,appMobil_tv3,5809111,...,CHROME,MOBILE,diferit,MzQ1MDQ2NzQw,NaN,False,MzQ1MDQ2NzQw,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2980050,True,2021-09-21 23:27:16,60,b8cb5320-da4a-11eb-971a-e3fd86aee49d,829171632259572094,False,IP,video,PortalCCMA,6120002,...,CHROME,MOBILE,diferit,MzQ1ODMyNzY0,NaN,True,MzQ1ODMyNzY0,9,21,23
2980051,True,2021-09-21 23:27:18,0,88d72a80-0bda-11ea-bdcf-d32cdc687e54,835391632259631354,False,IP,video,appTV_tv3,6118323,...,SAFARI,TV,diferit,MzI2NTE5NDQy,NaN,True,MzI2NTE5NDQy,9,21,23
2980052,True,2021-09-21 23:27:20,27,30b50120-1b13-11ec-a51e-3d25c93f171e,541771632259650113,False,IP,video,PortalCCMA,6120006,...,CHROME,PC,diferit,MzQ1MTUwNzg0,NaN,False,MzQ1MTUwNzg0,9,21,23
2980053,True,2021-09-21 23:27:43,120,db8526d0-1b21-11ec-bee3-a178ccf276eb,892101632259411606,False,IP,video,appMobil_tv3,6118323,...,CHROME,MOBILE,diferit,MzMxMzYxMDM2,NaN,False,MzMxMzYxMDM2,9,21,23


In [3]:
df = df.sort_values(by='min_data',ascending=True)   # Order the data by the reproduction date-time.


## 1.1- Eliminar unitats de consum

Primer que tot, eliminem aquells samples (unitats de consum) basant-nos en dos criteris:

    1. Aquells samples on la columna "durada" del videoclip és null (NaN), és a dir, que no estan informats. Els valors NaN de   les altres columnes, per ara, els tractarem com una variable discreta més.
    2. Aquells samples que corresponguin a àudio, ja que ens centrem només en video (columna media_tipus=audio)
    
Finalment, per el moment, tractem tots els altres valors NaN com una categoria més.

In [4]:
# drop the NaN values of the "durada" column.
df=df.dropna(subset=['durada'])

# drop those samples with media_tipus=audio (corresponding to radio consumption)
df = df.drop(df[df['media_tipus'] == 'audio'].index)
df = df.drop(df[df['canal_nom'] == 'Catalunya Ràdio'].index)
df = df.drop(df[df['canal_nom'] == 'Catalunya Música'].index)
df = df.drop(df[df['canal_nom'] == 'iCat.cat'].index)

# Reset the indexs of the dataframe
df=df.reset_index(drop=True)

# The last step is to convert all NaN values into the string "nan", to take them into account as an additional "category".
df = df.fillna('nan')

## 1.2- Variables temporals (continues)

·Transformem la columna de la data i hora de l'inici de la reproducció a format "datatime" (2019/04/02 12:43:45).

·Transformem la durada del videoclip a format "timedelta" (0 days 05:03:05), i la convertim en segons totals.

·Seguidament creem tres columnes noves: una amb l'any, l'altre amb el dia (nom) i l'altre amb el mes (nom).

·Finalment passem la columna de l'ininci de la reproducció (min_data) a segons, contant a partir de l'inici del dia a les 00:00:00 (hh:mm:ss). 

·Creem també dues columnes noves, indicant l'estació de l'any i si és cap de setmana o entre setmana

In [5]:
# We've seen that some samples in the "durada" column have the "-" character (ex: 0-2:30:13) which give problems. 
# So first we delete all - characters.
df.durada=df.durada.str.replace('-', '')
df.usuari_id=df.usuari_id.str.replace('-','')

# durada to timedelta
df['durada'] = pd.to_timedelta(df['durada']) 

# duration of the videoclip to total seconds.
df['durada']= df['durada']/np.timedelta64(1, 's') 

# min_data to datetime
df['min_data'] = pd.to_datetime(df['min_data'], format = '%Y/%m/%d %H:%M:%S')  

df["year"] = df['min_data'].dt.year  # new column with the year
df['day'] = df['min_data'].dt.day_name() # new column with the day
df['day2']=df['min_data'].dt.day # new column with the day (number)
df['month']= df['min_data'].dt.month_name() # new column with the month


###  1.2.1 - Shift horari. Posem l'origen a les 4a.m.

Realitzem un shift horari per tenir l'origen del consum sobre les 4 a.m, on és mínim (ja que a les 00h encara és molt elevat)

In [6]:
import datetime

df['min_data']= df['min_data'] - datetime.timedelta(hours=4)


### 1.2.2 - Columnes estació de l'any i cap de setmana / entre setmana

Agrupem la columna dels dies de la setmana amb Cap de setmana (Dissabte-Diumenge) o entre setmana. I també els mesos en estacions de l'any

In [7]:


# Weekend (Saturday-Sunday) or Weekdays (Monday-Friday)
weekend=[]
for j in range(len(df['day'])):
    if df['day'][j]=='Saturday' or df['day'][j]=='Sunday':
        weekend.append('weekend')
    else:
        weekend.append('weekdays')
        

# Season of the year:  
# 21 December - 20 March (winter).   
# 21 March - 20 June (Spring). 
# 21 June - 20 September (Summer).
# 21 September - 20 December (autumn).
season=[]
for i in range(len(df['month'])):
    if df['month'][i]=='January' or df['month'][i]=='February':
        season.append('winter')
    if df['month'][i]=='December' and df['min_data'][i].day>=21:
        season.append('winter')
    if df['month'][i]=='March'  and df['min_data'][i].day<21:
        season.append('winter')
        
    if df['month'][i]=='April' or df['month'][i]=='May':
        season.append('spring')
    if df['month'][i]=='March'  and df['min_data'][i].day>=21:
        season.append('spring')
    if df['month'][i]=='June'  and df['min_data'][i].day<21:
        season.append('spring')
        
    if df['month'][i]=='July' or df['month'][i]=='August':
        season.append('summer')
    if df['month'][i]=='June'  and df['min_data'][i].day>=21:
        season.append('summer')
    if df['month'][i]=='September'  and df['min_data'][i].day<21:
        season.append('summer')
        
    if df['month'][i]=='October' or df['month'][i]=='November':
        season.append('autumn')
    if df['month'][i]=='September'  and df['min_data'][i].day>=21:
        season.append('autumn')
    if df['month'][i]=='December'  and df['min_data'][i].day<21:
        season.append('autumn')


#New columns for weekend and season

df['weekend']=weekend    
df['season']=season

In [8]:
# Initial reproduction time to total seconds (from the start of the day 00:00:00)
df.min_data =df.min_data .dt.hour * 3600 + df.min_data .dt.minute * 60 + df.min_data .dt.second

## 1.3- Selecció de columnes

Creem nou DataFrame amb les dades netejades i processades, amb les columnes que necessitem

In [9]:

df2 = pd.DataFrame(            
    {'min_data': df['min_data'].astype(float),
     'durada_consum': df['bi_segons_consum'].astype(float),
     'durada':df['durada'].astype(float),
     'usuari_id': df['usuari_id'],
     'player_id': df['player_id'].astype(str),
     'producte_id': df['producte_id'].astype(str),
     'contingut_id': df['contingut_id'].astype(str),
     'canal_nom': df['canal_nom'].astype(str),
     'programa_nom': df['programa_nom'].astype(str),
     'programa_capitol': df['programa_capitol'].astype(str),  
     'tematica': df['tematica'].astype(str),
     'dispositiu_accepta_cookies': df['dispositiu_accepta_cookies'].astype(str),
     'dispositiu_model':df['dispositiu_model'].astype(str),
     'dispositiu_vendor':df['dispositiu_vendor'].astype(str),
     'dispositiu_browser_nom':df['dispositiu_browser_nom'].astype(str),
     'dispositiu_tipus':df['dispositiu_tipus'].astype(str),  
     'year':df['year'].astype(str), 
     'day':df['day'].astype(str), 
     #'day2':df['day2'].astype(str), 
     'month':df['month'].astype(str), 
     #'weekend':df['weekend'].astype(str),  # only use weekend and season to change the day name into weekdays or weekend and
     #'season': df['season'].astype(str)    # the month to its corresponding season (winter,spring,summer,autumn)
    })


In [11]:
df2

,min_data,durada_consum,durada,usuari_id,player_id,producte_id,contingut_id,canal_nom,programa_nom,programa_capitol,tematica,dispositiu_accepta_cookies,dispositiu_model,dispositiu_vendor,dispositiu_browser_nom,dispositiu_tipus,year,day,month
0,72083.0,224.0,3425.0,cd59b5d0eadd11e78b917b6a96b37d13,532431546297264625,PortalCCMA,5808597,TV3,Howards End,4,FICCIÓ,True,ANDROID,nan,CHROME,TABLET,2019,Tuesday,January
1,72170.0,936.0,2760.0,ef3a5e305a3011e8b2856b1b1792d2d5,151871546297352716,PortalCCMA,5809807,TV3,Telenotícies,5214,ACTUALITAT,True,ANDROID,nan,CHROME,MOBILE,2019,Tuesday,January
2,72283.0,6821.0,5825.0,971bc1a0d61711e7997557f16f7e845d,94429,appMobil_tv3,5808127,TV3,Pel·lícules,1,FICCIÓ,True,ANDROID,HUAWEI,ANDROID BROWSER,MOBILE,2019,Tuesday,January
3,72288.0,0.0,49.0,58169d60b60c11e885daa1c496d066bd,16755,appMobil_tv3,5809111,TV3,TV3,nan,PROMOCIÓ CADENA,True,ANDROID,nan,CHROME,MOBILE,2019,Tuesday,January
4,72329.0,0.0,49.0,7c5460400d5011e9b7c3ab71693993b0,81949,appMobil_tv3,5809111,TV3,TV3,nan,PROMOCIÓ CADENA,False,ANDROID,nan,CHROME,MOBILE,2019,Tuesday,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444828,70036.0,60.0,95.0,b8cb5320da4a11eb971ae3fd86aee49d,829171632259572094,PortalCCMA,6120002,TV3,Telenotícies comarques,5795.0,ACTUALITAT,True,ANDROID,nan,CHROME,MOBILE,2021,Tuesday,September
2444829,70038.0,0.0,2757.0,88d72a800bda11eabdcfd32cdc687e54,835391632259631354,appTV_tv3,6118323,TV3,Crims,22.0,DIVULGACIÓ,True,UE43RU7405UXXC,SAMSUNG,SAFARI,TV,2021,Tuesday,September
2444830,70040.0,27.0,29.0,30b501201b1311eca51e3d25c93f171e,541771632259650113,PortalCCMA,6120006,TV3,Notícies 3/24,6569.0,MÓN,False,WINDOWS,nan,CHROME,PC,2021,Tuesday,September
2444831,70063.0,120.0,2757.0,db8526d01b2111ecbee3a178ccf276eb,892101632259411606,appMobil_tv3,6118323,TV3,Crims,22.0,DIVULGACIÓ,False,ANDROID,nan,CHROME,MOBILE,2021,Tuesday,September
